<a href="https://colab.research.google.com/github/greyhound101/Orbuculum/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import pandas as pd
import numpy as np
import glob
import os
from matplotlib import pyplot as plt
import cv2
from keras.optimizers import *
import numpy as np
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import random
import keras.backend as K
from keras.applications import *
from keras.layers import *
from keras.models import Model
import tensorflow as tf
from keras.activations import *
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
import keras
import ast
from keras.utils import to_categorical
class DataGenerator(keras.utils.Sequence):
  def __init__(self, series, batch_size=64):
                       # array of labels
    self.paths=[]
    for en,i in enumerate(list(series.index)):
        self.paths+=ast.literal_eval(series[i])
    self.batch_size   = batch_size   
    
  def __len__(self):
    return int(np.floor(len(self.paths) / self.batch_size))


  def __getitem__(self, index):
		# selects indices of data for next batch
    img = [self.paths[k] for k in range(index*self.batch_size,(index+1)*self.batch_size)]
    images=[cv2.imread(path) for path in img]
    labels=[int(path.split('/')[2].split('_')[-1]) for path in img]
        
    
    
    return (np.asarray(images)/255).astype('float32'), to_categorical(labels,43)
class SE(keras.layers.Layer):
    def __init__(self,factor,otr,**kwargs):
        super(SE, self).__init__(**kwargs)
        self.inr=int(otr/factor)
        self.otr=otr
        
    def get_config(self):
        base_config = super(SE, self).get_config()

    def build(self, input_shape):
        super(SE, self).build(input_shape)
        self.pool1=GlobalAveragePooling2D()
        self.compress=Dense(self.inr,activation='relu')
        self.expand=Dense(self.otr,activation='sigmoid')

    def call(self, img):
        y = self.pool1(img)
        y = self.compress(y)
        map = self.expand(y)
        map = tf.expand_dims(map,1)
        map = tf.expand_dims(map,2)
        return tf.math.multiply(img,map)
def load_model(last=True): 
  K.clear_session()   
  model = Sequential()
  model.add(Conv2D(filters=32, kernel_size=(5,5), input_shape=(30,30,3)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPool2D(pool_size=(2, 2)))
  model.add(Conv2D(filters=64, kernel_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(SE(4,64))
    
  model.add(Conv2D(filters=32, kernel_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPool2D(pool_size=(2, 2)))
  model.add(Conv2D(filters=64, kernel_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(SE(8,64))
    
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(rate=0.5))
  model.add(Dense(43, activation='softmax'))
  return model
model=load_model()
model.summary()
df=pd.read_csv('/content/split.csv')
train=DataGenerator(df['train'])
test=DataGenerator(df['test'])
validation=DataGenerator(df['validation'])  
model.compile(optimizer=Adam(1e-3,decay=1e-3), 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])
ln=len(train)
hist=model.fit_generator(train,epochs=50,steps_per_epoch=ln,validation_data=validation)
model.save_weights('weights.hdf5')
np.save('history.npy',hist.history)
pre=model.predict(test)
np.save('soft_predictions.npy',pre)
pre=np.argmax(pre,1)
np.save('hard_predictions.npy',pre)
tot=[]
for i in range(len(test)):
    tot.append(test[i][-1].reshape(-1,43))
act=np.stack(tot).reshape(-1,43)
np.save('actual.npy',act)
print(accuracy_score(pre,np.argmax(act,1)))
plt.plot(hist.history['loss'])
plt.savefig('loss.png')
plt.show()
plt.plot(hist.history['accuracy'])
plt.savefig('accuracy.png')
plt.show()
plt.plot(hist.history['val_loss'])
plt.savefig('val_loss.png')
plt.show()
plt.plot(hist.history['val_accuracy'])
plt.savefig('val_accuracy.png')
plt.show()
